In [13]:
from dotenv import load_dotenv
load_dotenv()

import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [14]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '67ace808e558ca56a6e14d2d9fe97f68', 'Date': 'Wed, 21 Aug 2024 23:49:20 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [15]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Physics 101',
  'stars': 4,
  'review': 'Dr. Johnson explains concepts clearly and is very approachable. Her lectures are engaging, but the exams are a bit tough.'},
 {'professor': 'Professor Mark Thompson',
  'subject': 'Computer Science 201',
  'stars': 5,
  'review': 'Professor Thompson is an amazing instructor! His classes are well-structured, and he provides plenty of resources to help you succeed.'},
 {'professor': 'Dr. Emily Brown',
  'subject': 'Mathematics 301',
  'stars': 3,
  'review': 'Dr. Brown is knowledgeable but sometimes hard to follow. The course material is challenging, and the grading is strict.'},
 {'professor': 'Professor James Lee',
  'subject': 'History 101',
  'stars': 4,
  'review': 'Professor Lee is passionate about history and makes the subject come alive. His lectures are interesting, though the reading load is heavy.'},
 {'professor': 'Dr. Sarah Wilson',
  'subject': 'Chemistry 202',
  'stars': 2,
  'review

In [16]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [17]:
processed_data[0]

{'values': [-0.015181443,
  -0.021980805,
  0.03552888,
  0.016650205,
  0.020879231,
  -0.0013595559,
  -0.00042179468,
  0.04122667,
  -0.00052348437,
  0.01868875,
  0.020119527,
  -0.017498543,
  -0.031730354,
  0.014143179,
  -0.014130517,
  0.01768847,
  -0.028564917,
  -0.022829141,
  0.013826636,
  0.04006179,
  0.027703917,
  -0.018220264,
  0.042391554,
  -0.024842361,
  -0.03618729,
  -0.03514903,
  0.018903999,
  0.01630834,
  0.028311681,
  0.0010485515,
  0.08974651,
  -0.00027183202,
  -0.017587176,
  -0.031502444,
  -0.03428803,
  0.0492289,
  0.009388691,
  0.03443997,
  0.023322951,
  0.017169338,
  -0.014371091,
  0.019651042,
  -0.03725088,
  -0.022727849,
  0.04150523,
  0.007122237,
  0.0041815443,
  -0.028185064,
  0.035225,
  0.05206513,
  -0.046468638,
  0.021740232,
  0.044797286,
  -0.0044094557,
  -0.03471853,
  -0.025323508,
  0.014586341,
  0.026387095,
  0.003883993,
  -0.020727292,
  0.06305554,
  -0.03127453,
  -0.025437463,
  -0.016358986,
  -0.0244371

In [18]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}